# 基础知识
huggingface是一个平台，本身提供了一个transformers的底层机器学习库，同时还提供了很多开源的models和datasets。我们主要用到了里面的预训练模型，开箱即用，可以直接用到我们的工作中去。

## 模型model
huggingface上面有很多模型，包括预训练的模型，主要是解决NLP和图像以及音频类的任务。
这些任务叫做[task](https://huggingface.co/tasks)，每一个都有完整的说明和适用的场景。

### NLP任务列表
- Conversational
- Fill-Mask
- Question Answering
- Sentence Similarity
- Summarization
- Table Question Answering
- Text Classification
- Text Generation
- Token Classification
- Translation
- Zero-Shot Classification
[主要的NLP任务](https://huggingface.co/learn/nlp-course/zh-CN/chapter7/1?fw=pt)
[开箱即用的 pipelines](https://transformers.run/intro/2021-12-08-transformers-note-1/)

## transformers
pipeline是最简单的对象，第一个参数是task，第二个参数是model，可以只提供task，或者只提供model。task包含如下：
- feature-extraction （获得文本的向量化表示）
- fill-mask （填充被遮盖的词、片段）
- ner（命名实体识别）
- question-answering （自动问答）
- sentiment-analysis （情感分析）
- summarization （自动摘要）
- text-generation （文本生成）
- translation （机器翻译）
- zero-shot-classification （零训练样本分类）
由于每一个默认的都是用的英文，所以我们在公司使用的时候基本都要考虑一下支持中文的model。

In [1]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
cachedir='d:/huggingface_cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = cachedir
os.environ['TRANSFORMERS_CACHE'] = cachedir
os.environ['HF_HOME'] = cachedir

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("I've been waiting for a HuggingFace course my whole life.")
print(result)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598050713539124}]


上面默认的通常都是英文，如果需要支持中文的话，需要到官网进行筛选提取，比如可以使用`lxyuan/distilbert-base-multilingual-cased-sentiments-student`

In [3]:
from transformers import pipeline

classifier = pipeline(model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", return_all_scores=True)
result = classifier("我喜欢huggingface")
print(result)

result = classifier("我讨厌huggingface")
print(result)

Downloading:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

C:\Users\zhaowu.BMH\Anaconda3\lib\site-packages\transformers\pipelines\text_classification.py:89: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'positive', 'score': 0.9327275156974792}, {'label': 'neutral', 'score': 0.05581485107541084}, {'label': 'negative', 'score': 0.011457622051239014}]]
[[{'label': 'positive', 'score': 0.04255858436226845}, {'label': 'neutral', 'score': 0.09783820062875748}, {'label': 'negative', 'score': 0.859603226184845}]]


# 任务场景

## 不要每次都去下载配置，只做离线使用
设置model_kwargs中的local_files_only参数就行：

In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model_kwargs={'local_files_only':True})
result = classifier("I've been waiting for a HuggingFace course my whole life.")
print(result)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598050713539124}]


## 如何设置缓存目录？
可以通过环境变量来设置：
```
os.environ['HUGGINGFACE_HUB_CACHE'] = 'd:/huggingface_cache'
```

# 常见问题

## 在pipeline中的task可以是sentiment-analysis，但是在[官方文档任务描述](https://huggingface.co/tasks)中确没有，只有Text Classification，他们是什么样的对应关系？

在`transformers\pipelines\__init__.py`文件中可以找到如下代码：
```python
TASK_ALIASES = {
    "sentiment-analysis": "text-classification",
    "ner": "token-classification",
    "vqa": "visual-question-answering",
}
SUPPORTED_TASKS = {
    "audio-classification": {
        "impl": AudioClassificationPipeline,
        "tf": (),
        "pt": (AutoModelForAudioClassification,) if is_torch_available() else (),
        "default": {"model": {"pt": ("superb/wav2vec2-base-superb-ks", "372e048")}},
        "type": "audio",
    },
```
可以看到SUPPORTED_TASKS的key代表了所有可以选择的任务类型，其中的type对应类型，有audio/text/image/multimodal四种，default对应默认的模型名称。

TASK_ALIASES是一个别名，可以看到"sentiment-analysis"就对应"text-classification"，是一个东西。同样的，ner也对应token-classification。一个是人类容易理解的，另一个是程序的编号。
